In [37]:
import torch
from resnet9 import ResNet9
from opacus import PrivacyEngine
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, Normalize, ToTensor
from torch.utils.data import DataLoader

In [35]:
def get_cifar_dataloader(bs_train=256, bs_val=256):
    cifar10_mean = (0.4914, 0.4822, 0.4465)
    cifar10_std = (0.2471, 0.2435, 0.2616)

    train_ds = CIFAR10(
        "./data",
        train=True,
        download=True,
        transform=Compose([ToTensor(), Normalize(cifar10_mean, cifar10_std)]),
    )
    val_ds = CIFAR10(
        "./data",
        train=False,
        transform=Compose([ToTensor(), Normalize(cifar10_mean, cifar10_std)]),
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=bs_train,
        shuffle=True,
        num_workers=8,
        pin_memory=True,
        prefetch_factor=10,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=bs_val,
        shuffle=False,
        num_workers=8,
        pin_memory=True,
        prefetch_factor=10,
    )
    return train_loader, val_loader

In [38]:
file_path = "./models/cifar10_resnet9_50_39.pth.tar"

checkpoint = torch.load(file_path)
print("Parameters used for training")
for key in checkpoint:
    if key not in ["state_dict", "optimizer"]:
        print(f"{key}: {checkpoint[key]}")

seed = 34

torch.backends.cudnn.deterministic = True
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader, val_loader = get_cifar_dataloader(bs_train=64, bs_val=64)

model = ResNet9(
    3,
    10,
    torch.nn.Mish,
    checkpoint["scale_norm"],
    checkpoint["norm_layer"],
    checkpoint["num_groups"]
).to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.NAdam(model.parameters(), lr=checkpoint["learning_rate"])

if checkpoint["privacy"]:
    privacy_engine = PrivacyEngine()
    model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        target_epsilon=checkpoint["target_epsilon"],
        target_delta=checkpoint["target_delta"],
        epochs=checkpoint["epochs"],
        max_grad_norm=checkpoint["grad_norm"],
        noise_generator=torch.Generator(device=device).manual_seed(seed)
    )

model.load_state_dict(checkpoint["state_dict"])


Parameters used for training
act_func: <class 'torch.nn.modules.activation.Mish'>
batch_size: 1024
dataset: cifar
device: cuda
epochs: 50
final_epsilon: 2.8947377504325553
grad_norm: 1.5
group_norm: False
image_size: None
in_channels: 3
learning_rate: 0.001
log_interval: 20
max_batch_size: 128
model_arch: resnet9
noise_mult: None
norm_layer: group
num_classes: 10
num_groups: (32, 32, 32, 32)
privacy: True
scale_norm: True
secure_rng: False
target_delta: 1e-05
target_epsilon: 3
Files already downloaded and verified


<All keys matched successfully>

In [39]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in val_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += torch.nn.CrossEntropyLoss(reduction="sum")(
            output, target
        ).item()  # sum up batch loss
        pred = output.argmax(
            dim=1, keepdim=True
        )  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(val_loader.dataset)

print(
    "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)".format(
        test_loss,
        correct,
        len(val_loader.dataset),
        100.0 * correct / len(val_loader.dataset),
    )
)


Test set: Average loss: 1.3046, Accuracy: 6547/10000 (65.47%)
